In [1]:
import numpy as np
import pandas as pd

Read CSV and create Total Language Speakers DataFrame:

In [15]:
# Step 1 - Read CSV and filter for total language speakers
file_path = r"..\..\..\..\data\census\98-401-X2016066_English_CSV_data.csv"

df_total = pd.read_csv(file_path)

# Filter for total language speakers
df_total = df_total[
    df_total["DIM: Profile of Census Subdivisions (2247)"] 
    == "Total - Knowledge of official languages for the total population excluding institutional residents - 100% data"
]

print(df_total.shape)
print(df_total.head())

(575, 15)
      CENSUS_YEAR  GEO_CODE (POR)  GEO_LEVEL             GEO_NAME   GNR  \
99           2016         3501005          1      South Glengarry   3.9   
2346         2016         3501007          1  Akwesasne (Part) 59  34.1   
4593         2016         3501011          1       South Stormont   3.2   
6840         2016         3501012          1             Cornwall   4.2   
9087         2016         3501020          1         South Dundas   4.3   

      GNR_LF  DATA_QUALITY_FLAG CSD_TYPE_NAME  ALT_GEO_CODE  \
99       2.7                  0            TP       3501005   
2346    22.4               3323           IRI       3501007   
4593     5.9                  0            TP       3501011   
6840     4.0                  0            CY       3501012   
9087     7.0                  0            MU       3501020   

             DIM: Profile of Census Subdivisions (2247)  \
99    Total - Knowledge of official languages for th...   
2346  Total - Knowledge of official langua

Step 2 - Filter columns and rename for total speakers:

In [16]:
# Filter for relevant columns
df_total = df_total[["GEO_CODE (POR)", "Dim: Sex (3): Member ID: [1]: Total - Sex"]]

# Rename columns
df_total = df_total.rename(columns={
    "GEO_CODE (POR)": "GEOCODE",
    "Dim: Sex (3): Member ID: [1]: Total - Sex": "Total"
})

print(df_total.head())

      GEOCODE     Total
99    3501005  13055.00
2346  3501007   1470.00
4593  3501011  13000.00
6840  3501012  45625.00
9087  3501020  10810.00


Filter invalid values and convert to numeric:

In [17]:
# Filter out invalid values
invalid_values = ["x", "F", ".."]
df_total = df_total[~df_total["Total"].isin(invalid_values)]

# Convert to numeric
df_total["Total"] = pd.to_numeric(df_total["Total"], errors="coerce")

print(df_total.head())
print(df_total.dtypes)

      GEOCODE    Total
99    3501005  13055.0
2346  3501007   1470.0
4593  3501011  13000.0
6840  3501012  45625.0
9087  3501020  10810.0
GEOCODE      int64
Total      float64
dtype: object


Set GEOCODE as index:

In [18]:
# Step 3 - Set GEOCODE as index
df_total = df_total.set_index("GEOCODE")

print(df_total.head())
print(df_total.shape)

           Total
GEOCODE         
3501005  13055.0
3501007   1470.0
3501011  13000.0
3501012  45625.0
3501020  10810.0
(537, 1)


Create Bilingual (French and English) DataFrame:

In [19]:
# Step 1 - Read CSV and filter for bilingual people
df_bilingual = pd.read_csv(file_path)

# Filter for English and French speakers
df_bilingual = df_bilingual[
    df_bilingual["DIM: Profile of Census Subdivisions (2247)"] 
    == "English and French"
]

print(df_bilingual.shape)
print(df_bilingual.head())

(4025, 15)
     CENSUS_YEAR  GEO_CODE (POR)  GEO_LEVEL         GEO_NAME  GNR  GNR_LF  \
102         2016         3501005          1  South Glengarry  3.9     2.7   
107         2016         3501005          1  South Glengarry  3.9     2.7   
376         2016         3501005          1  South Glengarry  3.9     2.7   
645         2016         3501005          1  South Glengarry  3.9     2.7   
656         2016         3501005          1  South Glengarry  3.9     2.7   

     DATA_QUALITY_FLAG CSD_TYPE_NAME  ALT_GEO_CODE  \
102                  0            TP       3501005   
107                  0            TP       3501005   
376                  0            TP       3501005   
645                  0            TP       3501005   
656                  0            TP       3501005   

    DIM: Profile of Census Subdivisions (2247)  \
102                         English and French   
107                         English and French   
376                         English and French   
6

Filter columns and rename for bilingual:

In [20]:
# Filter for relevant columns
df_bilingual = df_bilingual[["GEO_CODE (POR)", "Dim: Sex (3): Member ID: [1]: Total - Sex"]]

# Rename columns
df_bilingual = df_bilingual.rename(columns={
    "GEO_CODE (POR)": "GEOCODE",
    "Dim: Sex (3): Member ID: [1]: Total - Sex": "French and English"
})

print(df_bilingual.head())

     GEOCODE French and English
102  3501005            6900.00
107  3501005              95.00
376  3501005             205.00
645  3501005             225.00
656  3501005              15.00


Filter invalid values and convert to numeric:

In [21]:
# Filter out invalid values
invalid_values = ["x", "F", ".."]
df_bilingual = df_bilingual[~df_bilingual["French and English"].isin(invalid_values)]

# Convert to numeric
df_bilingual["French and English"] = pd.to_numeric(df_bilingual["French and English"], errors="coerce")

print(df_bilingual.head())
print(df_bilingual.dtypes)

     GEOCODE  French and English
102  3501005              6900.0
107  3501005                95.0
376  3501005               205.0
645  3501005               225.0
656  3501005                15.0
GEOCODE                 int64
French and English    float64
dtype: object


Set GEOCODE as index for bilingual:

In [22]:
# Step 3 - Set GEOCODE as index
df_bilingual = df_bilingual.set_index("GEOCODE")

print(df_bilingual.head())
print(df_bilingual.shape)

         French and English
GEOCODE                    
3501005              6900.0
3501005                95.0
3501005               205.0
3501005               225.0
3501005                15.0
(3767, 1)


Merge DataFrames and calculate percentage:

In [23]:
# Merge both DataFrames on GEOCODE index
df_merged = pd.merge(df_total, df_bilingual, left_index=True, right_index=True)

print(df_merged.head())
print(df_merged.shape)

           Total  French and English
GEOCODE                             
3501005  13055.0              6900.0
3501005  13055.0                95.0
3501005  13055.0               205.0
3501005  13055.0               225.0
3501005  13055.0                15.0
(3759, 2)


Calculate bilingual percentage:

In [24]:
# Calculate percentage of bilingual people
df_merged["Bilingual Percentage"] = (df_merged["French and English"] / df_merged["Total"]) * 100

print(df_merged.head())
print(df_merged.describe())

           Total  French and English  Bilingual Percentage
GEOCODE                                                   
3501005  13055.0              6900.0             52.853313
3501005  13055.0                95.0              0.727691
3501005  13055.0               205.0              1.570280
3501005  13055.0               225.0              1.723478
3501005  13055.0                15.0              0.114899
              Total  French and English  Bilingual Percentage
count  3.759000e+03         3759.000000           3759.000000
mean   2.478813e+04          466.930035              1.770205
std    1.356943e+05         7256.411596              6.926920
min    4.500000e+01            0.000000              0.000000
25%    6.200000e+02            0.000000              0.000000
50%    2.985000e+03            0.000000              0.000000
75%    1.087500e+04           25.000000              0.357675
max    2.704415e+06       347510.000000             75.657895


Summary Statistics:

In [25]:
# Summary statistics for bilingual percentage
print("Summary Statistics for Bilingual Percentage:")
print(df_merged["Bilingual Percentage"].describe())
print("\nMin:", df_merged["Bilingual Percentage"].min())
print("Max:", df_merged["Bilingual Percentage"].max())
print("Mean:", df_merged["Bilingual Percentage"].mean())
print("Median:", df_merged["Bilingual Percentage"].median())
print("Std Dev:", df_merged["Bilingual Percentage"].std())

Summary Statistics for Bilingual Percentage:
count    3759.000000
mean        1.770205
std         6.926920
min         0.000000
25%         0.000000
50%         0.000000
75%         0.357675
max        75.657895
Name: Bilingual Percentage, dtype: float64

Min: 0.0
Max: 75.6578947368421
Mean: 1.770205451576506
Median: 0.0
Std Dev: 6.926920197425893


Drop duplicates for bilingual DataFrame:

In [27]:
# Drop duplicate GEOCODE index values, keeping the first occurrence
df_bilingual = df_bilingual[~df_bilingual.index.duplicated(keep='first')]

print(f"Bilingual DataFrame shape after drop_duplicates: {df_bilingual.shape}")
print(df_bilingual.head())

Bilingual DataFrame shape after drop_duplicates: (541, 1)
         French and English
GEOCODE                    
3501005              6900.0
3501007                50.0
3501011              4385.0
3501012             19030.0
3501020              1400.0


Re-merge DataFrames:

In [28]:
# Merge both DataFrames on GEOCODE index using inner join
df_merged = pd.merge(df_total, df_bilingual, left_index=True, right_index=True)

print("Merged DataFrame:")
print(df_merged.head())
print(f"Merged DataFrame shape: {df_merged.shape}")

Merged DataFrame:
           Total  French and English
GEOCODE                             
3501005  13055.0              6900.0
3501007   1470.0                50.0
3501011  13000.0              4385.0
3501012  45625.0             19030.0
3501020  10810.0              1400.0
Merged DataFrame shape: (537, 2)


Convert data types and calculate percentage:

In [29]:
# Convert to float64 to ensure proper division
df_merged["Total"] = df_merged["Total"].astype("float64")
df_merged["French and English"] = df_merged["French and English"].astype("float64")

# Calculate percentage of bilingual people
df_merged["Bilingual Percentage"] = (df_merged["French and English"] / df_merged["Total"]) * 100

print(df_merged.head())
print(df_merged.describe())

           Total  French and English  Bilingual Percentage
GEOCODE                                                   
3501005  13055.0              6900.0             52.853313
3501007   1470.0                50.0              3.401361
3501011  13000.0              4385.0             33.730769
3501012  45625.0             19030.0             41.709589
3501020  10810.0              1400.0             12.950971
              Total  French and English  Bilingual Percentage
count  5.370000e+02          537.000000            537.000000
mean   2.478813e+04         2775.465549             10.999332
std    1.358028e+05        18940.169990             15.303475
min    4.500000e+01            0.000000              0.000000
25%    6.200000e+02           30.000000              3.409091
50%    2.985000e+03          220.000000              5.748098
75%    1.087500e+04         1025.000000              9.464286
max    2.704415e+06       347510.000000             75.657895


Generate formatted summary statistics:

In [30]:
# Create a formatted summary statistics report
stats_min = df_merged["Bilingual Percentage"].min()
stats_max = df_merged["Bilingual Percentage"].max()
stats_mean = df_merged["Bilingual Percentage"].mean()
stats_median = df_merged["Bilingual Percentage"].median()
stats_std = df_merged["Bilingual Percentage"].std()

stats_report = f"""BILINGUAL POPULATION STATISTICS BY CENSUS SUBDIVISION (CSD)
============================================================

Minimum Percentage:          {stats_min:.2f}%
Maximum Percentage:          {stats_max:.2f}%
Mean Percentage:             {stats_mean:.2f}%
Median Percentage:           {stats_median:.2f}%
Standard Deviation:          {stats_std:.2f}%

Count:                       {df_merged["Bilingual Percentage"].count()}
25th Percentile (Q1):        {df_merged["Bilingual Percentage"].quantile(0.25):.2f}%
50th Percentile (Median):    {df_merged["Bilingual Percentage"].quantile(0.50):.2f}%
75th Percentile (Q3):        {df_merged["Bilingual Percentage"].quantile(0.75):.2f}%
"""

print(stats_report)

BILINGUAL POPULATION STATISTICS BY CENSUS SUBDIVISION (CSD)

Minimum Percentage:          0.00%
Maximum Percentage:          75.66%
Mean Percentage:             11.00%
Median Percentage:           5.75%
Standard Deviation:          15.30%

Count:                       537
25th Percentile (Q1):        3.41%
50th Percentile (Median):    5.75%
75th Percentile (Q3):        9.46%

